In [100]:
# Decimal scaled
# An attempt to implement multiprecision decimals in Python
# 2018-07-31    PV

import math

defaultDigits = 10

class Decimal():
    def __init__(self, value=0, digits=defaultDigits, isScaled=False):
        self.digits = digits
        self.scale = 10**digits
        if isScaled:
            self.value = value
        else:
            self.value = int(value*self.scale)
        
    def __str__(self):
        s = str(int(self.value//self.scale))
        r = self.value%self.scale
        if r>0:
            sr = str(r)
            s += '.' + '0'*(self.digits-len(sr)) + sr
        return s
    
    def __repr__(self):
        return "Decimal("+str(self)+", digits="+str(self.digits)+")"
    
    def scaledValue(self, digits=None):
        if digits:
            if digits>self.digits:
                return self.value*10**(digits-self.digits)
            else:
                return self.value//10**(self.digits-digits)
        else:
            return self.value
    
    
    def __abs__(self):
        return Decimal(abs(self.value), digits=self.digits, isScaled=True)

    def __neg__(self):
        return Decimal(-self.value, digits=self.digits, isScaled=True)

# direct arithmetic: Decimal <op> other
    def __add__(self, other):
        if not isinstance(other, Decimal): other=Decimal(other, digits=self.digits)
        di = max(self.digits, other.digits)
        return Decimal(self.scaledValue(di)+other.scaledValue(di), digits=di, isScaled=True)

    def __sub__(self, other):
        if not isinstance(other, Decimal): other=Decimal(other, digits=self.digits)
        di = max(self.digits, other.digits)
        return Decimal(self.scaledValue(di)-other.scaledValue(di), digits=di, isScaled=True)
    
    def __mul__(self, other):
        if not isinstance(other, Decimal): other=Decimal(other, digits=self.digits)
        di = max(self.digits, other.digits)
        return Decimal(self.scaledValue(di)*other.scaledValue(di)//10**di, digits=di, isScaled=True)

    def __truediv__(self, other):
        if not isinstance(other, Decimal): other=Decimal(other, digits=self.digits)
        di = max(self.digits, other.digits)
        return Decimal(10**di*self.scaledValue(di)//other.scaledValue(di), digits=di, isScaled=True)

# reverse arithmetic: other <op> Decimal
    def __radd__(self, other): return self+other

    def __rsub__(self, other): return -self+other
    
    def __rmul__(self, other): return self*other
    
    def __rtruediv__(self, other): return Decimal(other, digits=self.digits)/self

# Comparisons
# Don't implement __cmp__ since altough it's more compact, it's a bit less efficient
# Implementations of __ne__, __gt__ and __ge__ are implicit

    def __eq__(self, other):
            di = max(self.digits, other.digits)
            return self.scaledValue(di)==other.scaledValue(di)

    def __lt__(self, other):
            di = max(self.digits, other.digits)
            return self.scaledValue(di)<other.scaledValue(di)

    def __le__(self, other):
            di = max(self.digits, other.digits)
            return self.scaledValue(di)<=other.scaledValue(di)

# Square root
# Probably better as a class method
    def sqrt(self):
        # Start with a good approximation using math.sqrt
        r = Decimal(int(math.sqrt(self.value*self.scale)), digits=self.digits, isScaled=True)
        # Then quadratic convergence loop (Heron method)
        while True:
            r2 = (r+self/r)/2
            if r2==r: return r
            r = r2


d1 = Decimal(10, digits=50)
r1 = d1.sqrt()
print(r1)
print(r1*r1)

d2 = Decimal(10, digits=60)
r2 = d2.sqrt()
print(r2)
print(r2*r2)


3.16227766016837933199889354443271853371955513932521
9.99999999999999999999999999999999999999999999999995
3.162277660168379331998893544432718533719555139325216826857504
9.999999999999999999999999999999999999999999999999999999999994
